# Recommender Systems


recommmendation system: presents items to users in a relevant way

We will look at collaborative filtering

user: party that is receiving Recommendations
item: the passive party that is being recommended to users  

Content based vs collaborative filtering
In practice most will be hybrid.

## Some articles

"Factor in the Neighbors: Scalable and
Accurate Collaborative Filtering":
http://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf



## Recommendations at Expedia Group

### EPS

https://confluence.expedia.biz/pages/viewpage.action?pageId=890552932

https://www.dropbox.com/s/cf77o15jlahabay/wid-eps-recommendations.pdf

### BEX?

### Hcom?


In [1]:
import pandas as pd

data = [[1.0, 0.5, None, None], [None, 0.3], [0.4,0.9,0.8,0.7], [0.1]]
df = pd.DataFrame(data) 
df.rename(columns=lambda x: "item" + str(x), inplace=True)
df.rename(index = lambda x: "user" + str(x), inplace=True)
df

,item0,item1,item2,item3
user0,1.0,0.5,NaN,NaN
user1,NaN,0.3,NaN,NaN
user2,0.4,0.9,0.8,0.7
user3,0.1,NaN,NaN,NaN


The matrix will be sparse.

We want to fill in those user, item pairs using explicit and implicit interactions.

eg ratings, hearts, stars
watches, views, skips

From a practical standpoint this will often be the most work.




## Libraries

There are a number of libraries around.

Surprise is a Python scikit building and analyzing recommender systems.
http://surpriselib.com/
We'll be using it below as it's convenient for exploration

Also of interest tensorrec - hybrid recommender system
https://github.com/jfkirk/tensorrec


In [2]:
from surprise import SVD
from surprise import Dataset

from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy

import csv

data = Dataset.load_builtin('ml-100k')

def get_movies():
    movies = {}
    with open(data.ratings_file.replace("u.data","u.item"), 'r', encoding = "ISO-8859-1") as csvfile:
        moviesreader = csv.reader(csvfile, delimiter='|')
        for row in moviesreader:
            movies[int(row[0])] = row[1]
    return movies

def get_recs(algo,uid):
    recs = []
    for item in algo.trainset.all_items():
        pred = algo.predict(str(uid), str(item))
        recs.append((pred.est,item))
    recs.sort(reverse = True)
    return recs

def top_recs(i,recs):
    return list(map(lambda x: (x[0],movies[x[1]]), recs[:10]))

def get_ratings_by_user(uid):
    ratings = list(map(lambda x: tuple(reversed(x)), trainset.ur[uid]))
    ratings.sort(reverse = True)
    return ratings


def show_movie_ratings(ratings):
    return list(map(lambda x: (x[0],x[1],movies.get(x[1],x[1])), ratings))


In [3]:
movies = get_movies()
trainset, testset = train_test_split(data, test_size=.15)

## Collaborative Filtering - KNN

A collaborative filtering recommender system analyzes similarities between users and/or item interactions. Once the system identifies similarities, it serves users recommendations. In general, users see items that similar users liked.

https://surprise.readthedocs.io/en/stable/knn_inspired.html

### Measure Similarity
Pearson correlation coefficient, ρij, which measures the tendency of users to rate items i and j similarly
Others available including cosine Cosine

https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea


In [4]:
# k nearest neigbours
# user based
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
#algo = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})

algo.fit(trainset)
# now we can test predictions from the model against our testset
predictions = algo.test(testset)

# and we can test against test data
# mse -- mean sqare error
# rmse  -- root mean square error
# - how close fitted line is to data points

accuracy.rmse(predictions, verbose=True)



Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9388


0.938814854035541

### But what has been predicted?

In [5]:
show_movie_ratings(get_ratings_by_user(277))


[(5.0, 661, 'High Noon (1952)'),
 (5.0, 590, 'Hellraiser: Bloodline (1996)'),
 (5.0, 547, "Young Poisoner's Handbook, The (1995)"),
 (5.0, 536, 'Ponette (1996)'),
 (5.0, 384, 'Naked Gun 33 1/3: The Final Insult (1994)'),
 (5.0, 366, 'Dangerous Minds (1995)'),
 (5.0, 355, 'Sphere (1998)'),
 (5.0, 273, 'Heat (1995)'),
 (5.0, 267, 'unknown'),
 (5.0, 265, 'Hunt for Red October, The (1990)'),
 (5.0, 263, 'Steel (1997)'),
 (5.0, 251, 'Shall We Dance? (1996)'),
 (5.0, 186, 'Blues Brothers, The (1980)'),
 (5.0, 162, 'On Golden Pond (1981)'),
 (5.0, 92, 'True Romance (1993)'),
 (5.0, 70, 'Four Weddings and a Funeral (1994)'),
 (5.0, 66, 'While You Were Sleeping (1995)'),
 (4.0, 810, 'Shadow, The (1994)'),
 (4.0, 626, 'So Dear to My Heart (1949)'),
 (4.0, 556, 'Wild Bill (1995)'),
 (4.0, 548, 'NeverEnding Story III, The (1994)'),
 (4.0, 522, 'Down by Law (1986)'),
 (4.0, 413, 'Tales from the Crypt Presents: Bordello of Blood (1996)'),
 (4.0, 402, 'Ghost (1990)'),
 (4.0, 348, 'Desperate Measures 

In [6]:
show_movie_ratings(get_recs(algo,277)[:20])

[(5, 1597, 'Romper Stomper (1992)'),
 (5, 1591, 'Duoluo tianshi (1995)'),
 (5, 1536, 'Aiqing wansui (1994)'),
 (5, 1499, 'Grosse Fatigue (1994)'),
 (5,
  1242,
  'Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)'),
 (5, 1233, 'Nénette et Boni (1996)'),
 (4.9800682919082035, 1166, 'Love & Human Remains (1993)'),
 (4.967573696145125, 1344, 'Story of Xinghua, The (1993)'),
 (4.954912197437114, 1268, 'Bitter Moon (1992)'),
 (4.953910750903232, 1642, "Some Mother's Son (1996)"),
 (4.905605686214109, 1388, 'Gabbeh (1996)'),
 (4.899800750658986,
  113,
  'Horseman on the Roof, The (Hussard sur le toit, Le) (1995)'),
 (4.881487850820939, 1512, 'World of Apu, The (Apur Sansar) (1959)'),
 (4.855092722020315, 1398, 'Anna (1996)'),
 (4.849795918367347, 1467, 'Saint of Fort Washington, The (1993)'),
 (4.848911940466794, 1080, 'Celestial Clockwork (1994)'),
 (4.79282622139765, 1653, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (4.779790881897529,
  1347,
  

In [7]:
show_movie_ratings(get_recs(algo,277)[-20:])

[(1.5363288569803095, 1087, 'Bloodsport 2 (1995)'),
 (1.5010874283204627, 1288, 'Denise Calls Up (1995)'),
 (1.4709703831825514, 1419, 'Highlander III: The Sorcerer (1994)'),
 (1.4653914384829956, 1001, 'Stupids, The (1996)'),
 (1.447242726877985, 1472, 'Visitors, The (Visiteurs, Les) (1993)'),
 (1.4117138457067737, 894, 'Home Alone 3 (1997)'),
 (1.3835438915778204, 767, 'Addiction, The (1995)'),
 (1.3643550370722402, 375, 'Showgirls (1995)'),
 (1.3071360135377117, 247, 'Turbo: A Power Rangers Movie (1997)'),
 (1.2966527216669217, 1162, 'Phat Beach (1996)'),
 (1.2281507504447577, 352, 'Spice World (1997)'),
 (1.1659639772358847, 832, 'Bogus (1996)'),
 (1.163788981632698, 424, 'Children of the Corn: The Gathering (1996)'),
 (1.032421424343669, 314, '3 Ninjas: High Noon At Mega Mountain (1998)'),
 (1, 1621, 'Butterfly Kiss (1995)'),
 (1, 1609, 'B*A*P*S (1997)'),
 (1, 1376, 'Meet Wally Sparks (1997)'),
 (1, 1337, 'Larger Than Life (1996)'),
 (1, 1319, 'Neon Bible, The (1995)'),
 (1, 1304,

### Item based Nearest Neighbours

In [8]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)

show_movie_ratings(get_recs(algo,277)[:20])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9176


[(5, 1653, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5, 1642, "Some Mother's Son (1996)"),
 (5, 1599, "Someone Else's America (1995)"),
 (5, 1536, 'Aiqing wansui (1994)'),
 (5, 1449, 'Pather Panchali (1955)'),
 (5, 1358, 'The Deadly Cure (1996)'),
 (5, 1293, 'Star Kid (1997)'),
 (5, 1201, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5, 1122, 'They Made Me a Criminal (1939)'),
 (5, 814, 'Great Day in Harlem, A (1994)'),
 (4.856148491879351, 1500, 'Santa with Muscles (1996)'),
 (4.784403669724771, 1443, '8 Seconds (1994)'),
 (4.759615384615385, 1431, 'Legal Deceit (1997)'),
 (4.7492718658936495, 318, "Schindler's List (1993)"),
 (4.578611131291662, 64, 'Shawshank Redemption, The (1994)'),
 (4.518272098507291, 1398, 'Anna (1996)'),
 (4.5, 1463, 'Boys, Les (1997)'),
 (4.461843054035858, 286, 'English Patient, The (1996)'),
 (4.457526695675112, 1233, 'Nénette et Boni (1996)'),
 (4.456107028051363, 1388, 'Gabbeh (1996)')]

In [9]:
list(map(lambda x: movies.get(x,x), algo.get_neighbors(1536,10)))

["Breakfast at Tiffany's (1961)",
 'City of Angels (1998)',
 'Leaving Las Vegas (1995)',
 'Abyss, The (1989)',
 'Exit to Eden (1994)',
 'Good, The Bad and The Ugly, The (1966)',
 'Clerks (1994)',
 'Rumble in the Bronx (1995)',
 'Lone Star (1996)',
 "City Slickers II: The Legend of Curly's Gold (1994)"]

## SVD - Singular Value Decomposition

Famously used in Netflix Challenge

Matrix decomposition, also known as matrix factorization, involves describing a given matrix using its constituent elements.
calculated via iterative numerical methods

SVD is a matrix factorization technique that is usually used to reduce the number of features of a data set by reducing space dimensions from N to K where K < N. 





### Reading
http://nicolas-hug.com/blog/matrix_facto_3
http://nicolas-hug.com/blog/matrix_facto_4



In [10]:
# https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/
from numpy import array
from scipy.linalg import svd
A = array([[1, 2], [3, 4], [5, 6]])
print(A)
U, s, VT = svd(A)
print(U)
print(s)
print(VT)

[[1 2]
 [3 4]
 [5 6]]
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
[9.52551809 0.51430058]
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


## But how does this help us?

The matrix factorization is done on the user-item ratings matrix. 

From a high level, matrix factorization can be thought of as finding 2 matrices whose product is the original matrix.

We are after an ideal matrix with all the recommendations filled in.

Find it using stochastic gradient descent.


In [11]:
algo = SVD(n_factors = 20)
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)

show_movie_ratings(get_recs(algo,277)[:20])


RMSE: 0.9330


[(4.577048623202934, 408, 'Close Shave, A (1995)'),
 (4.36828521280946, 169, 'Wrong Trousers, The (1993)'),
 (4.339024843672453,
  114,
  'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.3288606652826855, 483, 'Casablanca (1942)'),
 (4.316161635804153, 657, 'Manchurian Candidate, The (1962)'),
 (4.2854688052854595, 134, 'Citizen Kane (1941)'),
 (4.272662237636945, 479, 'Vertigo (1958)'),
 (4.270542491791574, 187, 'Godfather: Part II, The (1974)'),
 (4.251994338828015, 603, 'Rear Window (1954)'),
 (4.239306250950143, 64, 'Shawshank Redemption, The (1994)'),
 (4.232729819802418, 641, 'Paths of Glory (1957)'),
 (4.224028901690533, 318, "Schindler's List (1993)"),
 (4.207139305671184, 285, 'Secrets & Lies (1996)'),
 (4.205647517607904, 178, '12 Angry Men (1957)'),
 (4.204854469925412, 515, 'Boot, Das (1981)'),
 (4.203581002429655, 513, 'Third Man, The (1949)'),
 (4.1920480714218185, 923, 'Raise the Red Lantern (1991)'),
 (4.180846552216582, 511, 'Lawrence of Arabia (1962)'),


In [12]:
algo = SVD(n_factors = 10)
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)

show_movie_ratings(get_recs(algo,277)[:20])


RMSE: 0.9327


[(4.374586458756627, 408, 'Close Shave, A (1995)'),
 (4.368236257502689, 603, 'Rear Window (1954)'),
 (4.352468289588259, 64, 'Shawshank Redemption, The (1994)'),
 (4.306744432535693, 169, 'Wrong Trousers, The (1993)'),
 (4.302838253600799, 318, "Schindler's List (1993)"),
 (4.280239471782148, 483, 'Casablanca (1942)'),
 (4.276697771416615, 12, 'Usual Suspects, The (1995)'),
 (4.255221349984832, 515, 'Boot, Das (1981)'),
 (4.252198037459238, 480, 'North by Northwest (1959)'),
 (4.2462052864694755, 513, 'Third Man, The (1949)'),
 (4.227238588632585,
  114,
  'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.210464789636268, 357, "One Flew Over the Cuckoo's Nest (1975)"),
 (4.195595994279022, 427, 'To Kill a Mockingbird (1962)'),
 (4.195401314756337, 657, 'Manchurian Candidate, The (1962)'),
 (4.195168941694706, 641, 'Paths of Glory (1957)'),
 (4.194643998617053, 170, 'Cinema Paradiso (1988)'),
 (4.16851463893154, 178, '12 Angry Men (1957)'),
 (4.167900665863122, 187, 'Godfa